# **Projet Python 2A**
Jean Baptiste Laval - Sirine Louati - Hadrien Lolivier

*Ce projet est réalisé dans le cadre du cours de python de Lino Galiana pour l'année 2021-2022.*

<div align="center">
  <img src="https://www.planete-energies.com/sites/default/files/styles/media_full_width_940px/public/thumbnails/image/visuel_emissions_co2_small.jpg?itok=Rrh1f3Qy"><br>
</div>
-----------------

## Introduction

**L'objectif** de notre programme est de retourner l'empreinte carbone de différentes recettes d'un plat choisi à la discrétion de  l'utilisateur. Ainsi, en rentrant le nom d'un plat, notre programme a pour mission de rendre compte de l'impact carbone de ce dernier à travers differents indicateurs et différentes recettes. Notre algorithme se décompose en trois grandes étapes :
    Etape 1: Implémentation d'un scrapper 
    Etape 2: Extraction de données et calcul de l'empreinte carbone des récettes
    Etape 3: Visualisation des résultats



## Les importations

Nous travaillons à partir du fichier main qui synthétise toutes les étapes de notre programme.



In [17]:
from main import main

## Etape 1: Implémentation d'un scrapper 

Nous présentons à l'aide d'un exemple simple l'utilisation du scrapper marmiton.

*Si l'utiliseur rentre "Gâteau au chocolat" que renvoie notre scrapper ?*

In [18]:
from src.dummy import dummy_output1
dummy_output1

{'Gâteau au chocolat fondant rapide': {'ingredients': {'chocolat pâtissier': 200.0,
   'farine': 50.0,
   'sucre en poudre': 100.0,
   'beurre': 100.0,
   'oeufs': 210.0},
  'nb_people': 6,
  'nb_comments': 1437,
  'mark': 4.7,
  'url': 'https://www.marmiton.org/recettes/recette_gateau-au-chocolat-fondant-rapide_166352.aspx'},
 'Gâteau au chocolat des écoliers': {'ingredients': {'chocolat noir': 200.0,
   'sucre en poudre': 200.0,
   'farine': 100.0,
   'levure chimique': 11.0,
   'oeufs': 280.0,
   'beurre doux': 125.0},
  'nb_people': 8,
  'nb_comments': 1240,
  'mark': 4.7,
  'url': 'https://www.marmiton.org/recettes/recette_gateau-au-chocolat-des-ecoliers_20654.aspx'},
 'Petits gâteaux tout chocolat': {'ingredients': {'farine': 100.0,
   'sucre': 100.0,
   'oeufs': 210.0,
   'crème dessert au chocolat': 200.0,
   'beurre': 100.0,
   'moules à muffin ou des caissettes en papier': 160.0},
  'nb_people': 9,
  'nb_comments': 337,
  'mark': 4.8,
  'url': 'https://www.marmiton.org/recett

*Construction d'un  indicateur de confiance*

Pour évaluer la fiabilité des différentes recettes, nous avons décidé de construire un indicateur de confiance basé sur la note de la recette sur marmiton ainsi que sur le nombre de commentaires associés à la recette.

*Dificultés survenues*

L'une des premières difficultés survenues lors de l'implémentation du scrapper fut les blocages d'accès du site marmiton. Pour éviter ces blocages à répétitions nous avons du recourir à quelques astuces (on peut citer par exemple l'utilisation de firefox, de la fonction sleep() pour simuler un comportement utilisateur). Par ailleurs, nous avons du surmonter certaines difficultés inattendues comme l'apparition potenteille de cookie, de condition d'ulisations ou encore les mises à jours du sites qui peuvent rapidement rendre obsolètes notre scrapper (possibilité qui peut en partie etre éviter en utilisant de manière judicieuse les chemins du site).


*Limites du Scrapper*

Dans la partie scrapping de notre projet certaines simplifications ont été nécéssaires  pour mener à bien notre projet. Tout d'abord, nous avons fait le choix de nous restreindre aux recettes issu du site "Marmiton", du fait de la variété de recettes associées à un plat.
Cependant il aurait pu etre interressant d'utiliser des recettes de diffrénts sites mais pour des raisons de simplicité et de complexité algorithme nous nous sommes concentrés sur les recettes proposées par Marmiton.
De plus, notre scrapper se devait de renvoyer un dictionnaire dans lequel chaque ingrédients était associé à son poids correspondnat dans la recette. Il a donc fallu procéder à des conversions et à certaines simplications pour rendre compte du poids de certaines quantités qui ne sont exprimées en gramme, par exemple les ingrédients du type (' 1 sachet de levure ', ' 1 cuillers à café ' sont automatiquement asssocié à un poids de 11 grammes et 15 grammes).

## Etape 2: Extraction de données et calcul de l'empreinte carbone des récettes

In [19]:
from src.dummy import dummy_aggregated_data
dummy_aggregated_data

,recipes,ingredients,weights,nb_comments,mark,url,agribalyse_match,distance,dqr,emission_type,co2
0,Gâteau au chocolat fondant rapide,chocolat pâtissier,0.303030,1437,4.7,https://www.marmiton.org/recettes/recette_gate...,"Chocolat blanc, tablette",0.123810,2.29,agriculture_co2,18.453596
1,le nouveau gâteau au chocolat (coco-llant),chocolat pâtissier,0.178333,159,4.7,https://www.marmiton.org/recettes/recette_le-n...,"Chocolat blanc, tablette",0.123810,2.29,agriculture_co2,18.453596
2,Gâteau au chocolat fondant rapide,farine,0.075758,1437,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
3,Gâteau au chocolat des écoliers,farine,0.109170,1240,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
4,Petits gâteaux tout chocolat,farine,0.114943,337,4.8,https://www.marmiton.org/recettes/recette_peti...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
...,...,...,...,...,...,...,...,...,...,...,...
289,Gâteau au chocolat type 'brownie',chocolat noir à croquer,0.209424,166,4.8,https://www.marmiton.org/recettes/recette_gate...,"Chocolat noir fourré praliné, tablette",0.121212,3.34,consumption_co2,0.000000
290,Gâteau au chocolat de type brownie,chocolat noir pâtissier,0.062799,155,4.8,https://www.marmiton.org/recettes/recette_gate...,"Chocolat noir fourré praliné, tablette",0.121212,3.34,consumption_co2,0.000000
291,Gâteau au chocolat de type brownie,poudre de noisette,0.023550,155,4.8,https://www.marmiton.org/recettes/recette_gate...,"Poudre cacaotée ou au chocolat pour boisson, s...",0.306667,2.77,consumption_co2,0.000000
292,le nouveau gâteau au chocolat (coco-llant),cassonade,0.267499,159,4.7,https://www.marmiton.org/recettes/recette_le-n...,"Cassis, cru",0.225000,2.92,consumption_co2,0.099565


In [15]:
main()

WebDriverException: Message: Can not connect to the Service chromedriver


*But*
calcul empreinte carbone en les asociant à la bonne base agribalyse ...
Difficulté à surmonter associé les bons ingrédients ...

In [ ]:
from src.aggregate import match_all_ingredients, aggregate_data
match_all_ingredients

<function src.aggregate.match_all_ingredients(recipes: Dict, distance: Callable = <function personalized_distance at 0x7f9f0f069598>) -> pandas.core.frame.DataFrame>